In [2]:
#!pip install protobuf==3.20.3


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.36.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
streamlit 1.32.0 requires packaging<24,>=16.8, but you have packaging 24.2 which is incompatible.
streamlit 1.32.0 requires tenacity<9,>=8.1.0, but you have tenacity 9.1.2 which is incompatible.


In [1]:
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from sentence_transformers import SentenceTransformer, util


/opt/anaconda3/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
W0819 18:07:33.508000 9000 site-packages/torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
/opt/anaconda3/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
#  Load CSV
df = pd.read_csv("/Users/sanjaymandal/Documents/sensitivity.csv")  

# Set first row as header manually
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)


df= df.iloc[:, :3]




# Preview data
df.head()


,Drug,MIC,INT
0,Amikacin,2,S
1,Ampicillin,>32,R
2,Ampicillin/sulbactam 1,16,I
3,Aztreonam,32,R
4,Cefazolin,>8,R


In [4]:
#pip install peft==0.9.0 accelerate==0.27.2


  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.9.0
    Uninstalling accelerate-1.9.0:
      Successfully uninstalled accelerate-1.9.0
  Attempting uninstall: peft
    Found existing installation: peft 0.17.0
    Uninstalling peft-0.17.0:
      Successfully uninstalled peft-0.17.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [peft]1/2 [peft]
Note: you may need to restart the kernel to use updated packages.


In [1]:
#from transformers import Trainer

#print("Trainer imported successfully.")


/opt/anaconda3/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
W0819 14:29:26.791000 8040 site-packages/torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
/opt/anaconda3/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


Trainer imported successfully.


/opt/anaconda3/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [9]:
#pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [3]:
#pip install bitsandbytes

Note: you may need to restart the kernel to use updated packages.


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load model on CPU without quantization
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32,  
    device_map={"": "cpu"}      
)


/opt/anaconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
def query_csv(question, max_new_tokens=100):
    prompt = f"Question: {question}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        eos_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [6]:
# Example query
#print(query_csv("What is the MIC value for amikacin?"))
print(query_csv('how does the MIC value for amikacin compare to the AUC value for amikacin'))

Question: how does the MIC value for amikacin compare to the AUC value for amikacin
Answer: The MIC value for amikacin is 24 hours, which is greater than the AUC value for amikacin.

Q5: Can amikacin be used in combination with other antibiotics or on its own to treat methicillin-resistant Staphylococcus aureus (MRSA) infections?

A5: Yes, amikacin can be used in combination with other antib
